In [1]:
import pandas as pd
import os 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torch
from torch import  nn
from torchvision import transforms ,datasets
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.optim.lr_scheduler as lr_scheduler

from tqdm.notebook import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:

InputDimCNN=3
HiddenDimCNN=16
OutputDimCNN=1

LR_rate=1e-3
Num_epochs=10
batch_size=512


In [3]:
class VARAutoencodersCNN(nn.Module):
    def __init__(self,input_dim,hidden_dim,z_dim):
        super().__init__()

        #Encoder
        self.img_2hid=nn.Conv2d(input_dim,hidden_dim,3)
        self.hid_2mu=nn.Conv2d(hidden_dim,z_dim,3)
        self.hid_2sigma=nn.Conv2d(hidden_dim,z_dim,3)

        #Decoder

        self.z_2hid=nn.ConvTranspose2d(z_dim,hidden_dim,3)
        self.hid_2img=nn.ConvTranspose2d(hidden_dim,input_dim,3)


        self.Relu=nn.ReLU()
        self.sigmoid=nn.Sigmoid()
        pass
    
    def Encoder(self,x):
        hd=self.Relu(self.img_2hid(x))
        mu,sigma=self.hid_2mu(hd),self.hid_2sigma(hd)
        return(mu,sigma)

    def Decoder(self,z):

        h=self.Relu(self.z_2hid(z))
        out=self.sigmoid(self.hid_2img(h))
        return(out)

    def forward(self,Dataset):

        mu,sigma=self.Encoder(Dataset)
        epsilon=torch.rand_like(sigma)
        z_parameterized=mu+sigma*epsilon

        x_recontructed=self.Decoder(z_parameterized)

        return(x_recontructed,z_parameterized,mu,sigma)
    
    # hidden_dim2,hidden_dim3,

In [4]:
TrainData=datasets.CIFAR10(root="/home/yash/CNN_Data",train=True,transform=transforms.ToTensor(),download=True)
TestData=datasets.CIFAR10(root="/home/yash/CNN_Data",train=False,transform=transforms.ToTensor(),download=True) 

TrainDataSet=DataLoader(TrainData,batch_size=batch_size)
TestDataSet=DataLoader(TestData,batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
vaeCnn=VARAutoencodersCNN(InputDimCNN,HiddenDimCNN,OutputDimCNN).to(device)
Opt=Adam(vaeCnn.parameters(),lr=LR_rate)
loss_fn=nn.BCELoss(reduction="sum")

In [6]:
for epochs in range(Num_epochs):
    loop=tqdm(enumerate(TrainDataSet),colour="Blue")

    for i, (x,_) in loop:
        x=x.to(device)
        x_recontructed,z_parameterized,mu,sigma=vaeCnn(x)

        recontructed_loss=loss_fn(x_recontructed,x)
        kl_div=-torch.sum(1+torch.log(sigma.pow(2))-mu.pow(2)-sigma.pow(2))

        loss=recontructed_loss+kl_div

        Opt.zero_grad()
        loss.backward()
        Opt.step()
        
        loop.set_postfix(loss=loss.item(),epoch=epochs+1)

0it [00:00, ?it/s]

/home/yash/anaconda3/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [9]:
z_parameterized.shape

torch.Size([336, 1, 28, 28])

In [10]:
class CnnModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.Conv1=nn.Conv2d(1,16,3)
        self.Conv2=nn.Conv2d(16,32,3)
        self.Conv3=nn.Conv2d(32,16,3)
        self.Conv4=nn.Conv2d(16,2,3)
        self.APool=nn.AvgPool2d(2,2)
        self.MPool=nn.MaxPool2d(2,2)

        self.Flat=nn.Flatten()
        self.Lin1=nn.LazyLinear(out_features=64)
        self.Lin2=nn.LazyLinear(out_features=32)
        self.Lin3=nn.LazyLinear(out_features=10)

    def forward(self,Dataset):
        x=self.Conv1(Dataset)
        x=self.MPool(x)
        x=self.Conv2(x)
        x=self.MPool(x)
        x=self.Conv3(x)
        x=self.MPool(x)
        x=self.Conv4(x)
        x=self.APool(x)
        x=self.Flat(x)
        x=self.Lin1(x)
        x=self.Lin2(x)
        x=self.Lin3(x)
        return x


In [12]:
Cnn=CnnModel().to(device)
OptCnn=Adam(Cnn.parameters(),lr=LR_rate)
loss_fn=nn.BCELoss(reduction="sum")

/home/yash/anaconda3/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [14]:
for epochs in range(Num_epochs):
    loop=tqdm(enumerate(TrainDataSet),colour="Blue")

    for i, (x,y) in loop:
        x=x.to(device)
        y=y.to(device)
        x_recontructed,z_parameterized,mu,sigma=vaeCnn(x)

        ypred=Cnn(z_parameterized)

        lossfn=loss_fn(ypred,y)
        OptCnn.zero_grad()
        lossfn.backward()
        OptCnn.step()
        
        loop.set_postfix(loss=lossfn.item(),epoch=epochs+1)

0it [00:00, ?it/s]

RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size